In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression

In [2]:
import pickle

In [3]:
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

### data summary
- There are 68211 total records, after filtering by duration > 1 & <=60 there will be 65924 records, which is 96% of the data

In [10]:
def read_and_preprocess(filename):
    df = pd.read_parquet(filename)
    
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    
    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    
    df['duration'] = df['duration'].apply(lambda td: td.total_seconds()/60)
    df['PU_DO'] = df['PULocationID'].astype(str) + '_' + df['DOLocationID'].astype(str)
    
    #categorical = ['PU_DO']#'DOLocationID',]
    categorical = ['PULocationID','DOLocationID']#'DOLocationID',]
    
    df[categorical]= df[categorical].astype(str)
    df = df[(df.duration > 1) & (df.duration <=60)]
    return df

In [11]:
df_train = read_and_preprocess('../data/green_tripdata_2023-01.parquet')
df_valid = read_and_preprocess('../data/green_tripdata_2023-02.parquet')

In [12]:
len(df_train),len(df_valid)

(65924, 62547)

### Training pipeline

In [13]:
dv = DictVectorizer()

train_dict = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dict = df_valid[categorical + numerical].to_dict(orient='records')
x_val = dv.transform(val_dict)

In [14]:

target = 'duration'
y_train = df_train[target].values
y_valid = df_valid[target].values

In [15]:

#training
Lr = LinearRegression()
Lr.fit(X_train,y_train)
#prediction
y_pred = Lr.predict(x_val)
#evaluation
mean_squared_error(y_valid,y_pred, squared=False)

7.3517729073485345

##### dump model to pickle

In [18]:
with open('../models/lin_Reg.bin', 'wb') as f_out:
    pickle.dump((dv,Lr), f_out)

## with Lasso Reg

In [19]:
#With Lasso
lr = Lasso(alpha=0.01)
lr.fit(X_train,y_train)
#prediction
y_pred = lr.predict(x_val)
#evaluation
mean_squared_error(y_valid,y_pred, squared=False)

7.7361261061027395

In [20]:
#With Ridge
LR = Ridge()
LR.fit(X_train,y_train)
#prediction
y_pred = LR.predict(x_val)
#evaluation
mean_squared_error(y_valid,y_pred, squared=False)

7.348918521717934